# Immigration data

In [2]:
import pandas as pd
from IPython.display import Markdown;
import numpy

## immigration_citizenship.tsv

## immigration_country_born.tsv

In [62]:
url = "../../Data/immigration/immigration_country_born.tsv"
table_raw = pd.read_csv(url,sep='\t')
year_range = range(2008,2024,1)


In [64]:

age = []
agedef = []
birth_country = []
sex = []
country = []

for index,row in table_raw.iterrows():
    title_split = (row[0].split(","))
    # print(title_split)
    age.append(title_split[1]) 
    agedef.append(title_split[2])
    birth_country.append(title_split[3])
    sex.append(title_split[5])
    country.append(title_split[6])
    
    

table_raw["age"]= age
table_raw["agedef"] = agedef
table_raw["birth_country"] = birth_country
table_raw["sex"]=sex
table_raw["country"]=country

table_raw.drop(columns=["freq,age,agedef,c_birth,unit,sex,geo\TIME_PERIOD"],inplace=True)

<>:24: SyntaxWarning: invalid escape sequence '\T'
<>:24: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipykernel_21217/3666701640.py:24: SyntaxWarning: invalid escape sequence '\T'
  table_raw.drop(columns=["freq,age,agedef,c_birth,unit,sex,geo\TIME_PERIOD"],inplace=True)
/tmp/ipykernel_21217/3666701640.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  title_split = (row[0].split(","))


#### Reindexing, renaming columns, adding ISO3 codes


In [65]:
#rename country codes
country_codes = pd.read_csv("../../Data/country-region-codes.csv")
new_columns = {}

#numeric columns
for column in table_raw.columns:
    if column.strip().isnumeric():
        new_columns[column] = int(column.strip())
    else:
        new_columns[column]=column
        
table = table_raw.rename(columns=new_columns)
table.rename(columns={"country":"alpha-2"},inplace=True)

table = pd.merge(table,country_codes,on="alpha-2").drop(columns=["country-code","alpha-2"]).rename(columns={"name":"Country","alpha-3":"ISO3"})

#reordering
new_order = list(table.columns[-5:])+[i for i in year_range]

table = table[new_order]
# display(table.head())

#### Fixing values
Renaming : to NaN
fixing provisional data

In [66]:
replaced = table.copy()

#fix numeric values
for column in replaced.columns[3:]:
    replaced[column] = replaced[column].apply(str.strip)
    # print(type(replaced[column][0]))

replaced.replace(to_replace=":",value=numpy.nan,inplace=True)

#tags
tags = ["b","p","ep","e","be","bep","P"]

#I used Deepseek AI for this, it was quite complex
pattern = r'\b(?:' + '|'.join(tags) + r')\b'

# Apply the replacement to all columns (or specify specific columns)
df = replaced.apply(lambda x: x.str.replace(pattern, '', regex=True).str.strip())



#### Saving to file

In [67]:
path = "../../Data/immigration/immigration_country_born.csv"

df.to_csv(path,index=False)